In [1]:
import os
import csv
import glob
import shutil
import pandas as pd
from astropy.io import fits
from sunpy.io.header import FileHeader

In [2]:
harpnum_dir = "allharp2noaa2.csv"
main_dir = "/data/SHARPS/raw-sharps"
destination_folder = "/data/SHARP_HARP"
no_lat_values = "/data/SHARP_HARP/no_lat_files.csv"

In [3]:
folder_list = os.listdir(main_dir)
df = pd.read_csv(harpnum_dir)
harpnum_list = df["DEF_HARPNUM"].tolist()

In [4]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [5]:
def savefile(main_dir, foldername, magnetogram_file, destination_folder):
    source = os.path.join(main_dir, foldername, magnetogram_file)
    destination = os.path.join(destination_folder, foldername)
    if not os.path.isdir(destination):
        os.mkdir(destination)
    shutil.copy(source, destination)

In [6]:
common_folder_list = intersection(folder_list, list(map(str, harpnum_list)))
destination_folder_list = os.listdir(destination_folder)
for foldername in common_folder_list:
    folderpath = os.path.join(main_dir, foldername)
    files_inside_folder = listdir_nohidden(folderpath)
    magnetogram_files = [file for file in files_inside_folder if 'magnetogram' in file]

    for magnetogram_file in magnetogram_files:
        time = magnetogram_file.split("_")[3]
        if time in ['000000', '120000']:
            
            # for getting latlong 
            hdul = fits.open(magnetogram_file)
            hdr = hdul[1].header 
            if foldername not in destination_folder_list:
                try:
                    LONDTMIN, LONDTMAX = hdr['LONDTMIN'], hdr['LONDTMAX']
                    if LONDTMIN >= -70 and LONDTMAX <= 70:
                        savefile(main_dir, foldername, magnetogram_file, destination_folder)

                except:
                    with open(no_lat_values, "a") as f:                    
                        writer = csv.writer(f)
                        writer.writerow([magnetogram_file])